# Caso práctico U-NET con uso de LUNA dataset
El LUng Nodule Analysis 2016 (LUNA16) se centra en la detección de nódulos pulmonares a partir de tomografías computarizadas (CT) de dosis baja, con el objetivo de mejorar el diagnóstico temprano de cáncer de pulmón, la principal causa de muerte relacionada con cáncer en el mundo.


## Configuración e instalación de bibliotecas
Se realiza la configuración e instalación de las bibliotecas necesarias para llevar a cabo la implementación de la arquitectura U-Net en el análisis de imágenes del dataset LUNA16.


### Instalación de dependencias


In [ ]:
%pip install tensorflow
%pip install pillow
%pip install scikit-image
%pip install matplotlib
%pip install scikit-learn


### Importación de bibliotecas


In [ ]:
import tensorflow as tf  # Importa TensorFlow, usado para construir y entrenar la U-Net
import os  # Para operaciones del sistema de archivos, como listar directorios
import random  # Para la generación de números aleatorios (e.g., para divisiones aleatorias de datos)
import numpy as np  # Biblioteca de cálculo numérico eficiente


### Importación de módulos específicos


In [ ]:
from tqdm import tqdm  # Barra de progreso para bucles largos
from PIL import Image  # Manejo de imágenes
from skimage.io import imread, imshow  # Carga y visualización de imágenes
from skimage.transform import resize  # Redimensionado de imágenes
import matplotlib.pyplot as plt  # Visualización de datos
from sklearn.model_selection import train_test_split  # División del dataset en entrenamiento y prueba


### Importación de Keras y TensorFlow para modelos


In [ ]:
from keras import layers  # Capas utilizadas en la red neuronal
import keras  # Biblioteca de alto nivel para crear redes neuronales (encima de TensorFlow)
from IPython.display import Image, display  # Muestra imágenes en un notebook Jupyter
from keras.utils import load_img  # Carga de imágenes (similar a PIL)
from keras import utils  # Utilidades de Keras para el manejo de datos
from PIL import ImageOps  # Para aplicar operaciones de imagen (como reflejos, rotaciones)
from tensorflow import data as tf_data  # API de manejo de datos en TensorFlow
from tensorflow import image as tf_image  # API de operaciones específicas de imágenes en TensorFlow
from tensorflow import io as tf_io  # API para operaciones de entrada/salida
from tensorflow.keras import Model, models  # Clases base para la creación de modelos en Keras
from tensorflow.keras.applications import ResNet50  # Modelo preentrenado que se puede usar como base
from tensorflow.keras.optimizers import Adam  # Optimizador específico para el entrenamiento del modelo


## Preprocesamiento de datos
Comienza con la extracción de archivos comprimidos (en este caso, archivos .zip) que contienen las imágenes para el entrenamiento y la prueba.


### Extracción inicial de archivos


In [ ]:
from zipfile import ZipFile

# Extraer archivos del archivo ZIP a las carpetas correspondientes
with ZipFile("C:\\Users\\PC\\Downloads\\data-science-bowl-2018.zip", "r") as zip_ref:
    zip_ref.extractall("./stage1_train")  # Extraer a la carpeta de entrenamiento

# Extraer nuevamente el ZIP para separar datos de prueba, si es necesario
with ZipFile("C:\\Users\\PC\\Downloads\\data-science-bowl-2018.zip", "r") as zip_ref:
    zip_ref.extractall("./stage1_test")  # Extraer a la carpeta de prueba


### Función para extraer archivos ZIP de un directorio


In [ ]:
import os
from zipfile import ZipFile

# Funcion para extraer archivos ZIP
def extract_zip_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.zip'):
            zip_path = os.path.join(directory, filename)
            with ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(directory)


### Uso de la función de extracción


In [ ]:
#Extraer archivos ZIP de stage1_train
extract_zip_files('./stage1_train')

#Extraer archivos ZIP de stage1_test
extract_zip_files('./stage1_test')


### Listado de directorios


In [ ]:
print(os.listdir('./stage1_train'))
print(os.listdir('./stage1_test'))


## Preprocesamiento de imágenes
Se cubre especialmente el ajuste de tamaño, la normalización y la preparación de las máscaras para el entrenamiento del modelo U-Net.


### Dimensiones y configuraciones iniciales


In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

TRAIN_PATH = 'stage1_train/'
TEST_PATH = 'stage1_test/'

train_ids = [d for d in os.listdir(TRAIN_PATH) if os.path.isdir(os.path.join(TRAIN_PATH, d))]
test_ids = [d for d in os.listdir(TEST_PATH) if os.path.isdir(os.path.join(TEST_PATH, d))]

x = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)


### Verificación del contenido de las carpetas


In [ ]:
import os

# Verificar el contenido de las carpetas de entrenamiento y prueba
print("Contenido de la carpeta de entrenamiento:")
print(os.listdir('./stage1_train'))

print("Contenido de la carpeta de prueba:")
print(os.listdir('./stage1_test'))


### Redimensionamiento y preprocesamiento de imágenes de entrenamiento


In [ ]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm

print('Resizing training images and masks')
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img_path = os.path.join(path, 'images', f'{id_}.png')
    img = imread(img_path)

    # Verificar si la imagen es 2D (escala de grises) y convertir a 3D
    if img.ndim == 2:  # Escala de grises
        img = np.stack((img,)*3, axis=-1)  # Convertir a RGB duplicando el canal
    elif img.shape[2] > 3:  # Si hay más de 3 canales, tomar solo los primeros 3
        img = img[:, :, :3]  # Eliminar el canal alfa si existe
    elif img.shape[2] < 3:  # Si hay menos de 3 canales, puedes duplicar
        img = np.stack((img,)*3, axis=-1)

    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    x[n] = img  # Almacena la imagen redimensionada en x
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

    # Recorrer todas las subcarpetas en el directorio de máscaras
    masks_path = os.path.join(path, 'masks')
    for dirpath, dirnames, filenames in os.walk(masks_path):
        for mask_file in filenames:
            mask_ = imread(os.path.join(dirpath, mask_file))
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_)

    y[n] = mask  # Almacena la máscara en y


### Visualizacion de la estructura de las carpetas


In [ ]:
import os

# Listar contenido de la carpeta de entrenamiento
print("Contenido de la carpeta de entrenamiento:")
for root, dirs, files in os.walk('stage1_train'):
    print(f"Directorio: {root}")
    for file in files:
        print(f"  Archivo: {file}")
    for dir in dirs:
        print(f"  Subcarpeta: {dir}")

# Listar contenido de la carpeta de prueba
print("Contenido de la carpeta de prueba:")
for root, dirs, files in os.walk('stage1_test'):
    print(f"Directorio: {root}")
    for file in files:
        print(f"  Archivo: {file}")
    for dir in dirs:
        print(f"  Subcarpeta: {dir}")


### Visualización de IDs de entrenamiento y prueba


In [ ]:
print(train_ids)
print(test_ids)


### División del dataset en entrenamiento y prueba


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


### Verificación de las dimensiones del conjunto de datos


In [ ]:
x_train[0].shape
x_train.shape


### Visualización aleatoria de una imágen de entrenamiento y su máscara correspondiente


In [ ]:
image_x = random.randint(0, len(x_train))
plt.axis("off")
imshow(x_train[image_x])
plt.show()

plt.axis("off")
imshow(np.squeeze(y_train[image_x]))
plt.show()


### Preparación de imágenes de prueba


In [ ]:
test_images = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Resizing test images')

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = os.path.join(TEST_PATH, id_)
    img_path = os.path.join(path, 'images', f'{id_}.png')
    img = imread(img_path)

    # Verificar las dimensiones de la imagen
    if img.ndim == 2:  # Escala de grises
        img = np.stack((img,)*3, axis=-1)  # Convertir a RGB duplicando el canal
    elif img.shape[2] > 3:  # Si hay más de 3 canales, tomar solo los primeros 3
        img = img[:, :, :3]  # Eliminar el canal alfa si existe
    elif img.shape[2] < 3:  # Si hay menos de 3 canales, puedes duplicar
        img = np.stack((img,)*3, axis=-1)

    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    test_images[n] = img

print('Done!')


### Definición de parámetros de entrada para la U-Net


In [ ]:
input_shape = (128, 128, 3)
num_classes = 1


## Construcción de la U-Net
Esta sección contiene la implementación de la arquitectura U-Net para la segmentación de imágenes, además de la creación de un modelo ResNet. También hay partes dedicadas a la visualización de los resultados y predicciones con diferentes arquitecturas.


### Definición del modelo U-Net


In [ ]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

# Contraction path (Encoder)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
b1 = tf.keras.layers.BatchNormalization()(c1)
r1 = tf.keras.layers.ReLU()(b1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(r1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
b2 = tf.keras.layers.BatchNormalization()(c2)
r2 = tf.keras.layers.ReLU()(b2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(r2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
b3 = tf.keras.layers.BatchNormalization()(c3)
r3 = tf.keras.layers.ReLU()(b3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(r3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
b4 = tf.keras.layers.BatchNormalization()(c4)
r4 = tf.keras.layers.ReLU()(b4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(r4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
b5 = tf.keras.layers.BatchNormalization()(c5)
r5 = tf.keras.layers.ReLU()(b5)
c5 = tf.keras.layers.Dropout(0.3)(r5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)


### Decodificar (Expansive Path)


In [ ]:
# Expansive path (Decoder)
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
u6 = tf.keras.layers.BatchNormalization()(u6)
u6 = tf.keras.layers.ReLU()(u6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u6)
u7 = tf.keras.layers.concatenate([u7, c3])
u7 = tf.keras.layers.BatchNormalization()(u7)
u7 = tf.keras.layers.ReLU()(u7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u7)
u8 = tf.keras.layers.concatenate([u8, c2])
u8 = tf.keras.layers.BatchNormalization()(u8)
u8 = tf.keras.layers.ReLU()(u8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(u8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
u9 = tf.keras.layers.BatchNormalization()(u9)
u9 = tf.keras.layers.ReLU()(u9)

outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(u9)


### Compilación y entrenamiento del modelo U-Net


In [ ]:
model_unet = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model_unet.compile(optimizer=Adam(learning_rate=0.005), loss='binary_crossentropy', metrics=['accuracy'])
model_unet.summary()


### Callbacks y Entrenamiento 


In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]

model_unet.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=16, epochs=25, callbacks=callbacks)


## Visualización de resultados de entrenamiento


In [ ]:
accuracy = model_unet.history.history['accuracy']
val_accuracy = model_unet.history.history['val_accuracy']

plt.figure()
plt.plot(accuracy, 'r', label='Training accuracy')
plt.plot(val_accuracy, 'bo', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()


## Implementación del modelo ResNet


In [ ]:
model_resnet_tf = tf.keras.Sequential([
    ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model_resnet_tf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Visualización y predicción


In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 5))
    title = ['Input Image', 'Ground Truth', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

# Predicción de máscaras usando diferentes modelos y visualización
display([sample_image, sample_mask, predicted_mask_unet])


## Preparación de la muestra para la predicción
Realización de predicciones con diferentes modelos (U-Net y ResNet), así como en la visualización de los resultados de segmentación.


### Selección aleatoria de una imágen de prueba


In [ ]:
i = random.randint(0, len(x_test))
sample_image = x_test[i]
sample_mask = y_test[i]


## Predicción usando diferentes modelos


In [ ]:
# Lista de modelos para predecir
models = [model_unet, model_resnet, model_resnet_tf]

# Diccionario para almacenar predicciones
predictions = {}

# Iterar a través de cada modelo y hacer predicciones.
for model in models:
    model_name = model.__class__.__name__  # Obtener el nombre de clase del modelo.
    try:
        predictions[model_name] = model.predict(sample_image[tf.newaxis, ...])[0]
    except Exception as e:
        predictions[model_name] = f"Error: {str(e)}"  # Almacenar el mensaje de error


## Uso de modelos con nombres personalizados


In [ ]:
# Lista de modelos para predecir con sus nombres personalizados
model_info = {
    "U-Net": model_unet,
    "ResNet": model_resnet,
    "ResNetTF": model_resnet_tf,
}

# Diccionario para almacenar predicciones
predictions = {}

# Iterar a través de cada modelo y hacer predicciones.
for model_name, model in model_info.items():
    try:
        predictions[model_name] = model.predict(sample_image[tf.newaxis, ...])[0]
    except Exception as e:
        predictions[model_name] = f"Error: {str(e)}"  # Almacenar el mensaje de error

# Imprime las claves del diccionario de predicciones
print(predictions.keys())


## Instalación de OpenCV para visualización adicional


In [ ]:
%pip install opencv-python

import cv2

# Cargue la imagen (asegúrese de proporcionar una ruta válida)
sample_image = cv2.imread('path_to_image.jpg')

# Compruebe si la imagen está cargada correctamente
if sample_image is not None:
    sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)  # Convertir a RGB si es necesario
    print("Sample image shape:", sample_image.shape)
else:
    print("Error loading image!")


## Comprobación de formatos de las imágenes y visualización 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Check the shapes
print("Sample image shape:", sample_image.shape)
print("Sample mask shape:", sample_mask.shape)
print("Predicted mask U-Net shape:", predicted_mask_unet.shape)

# Asegúrese de que predicted_mask_unet tenga la forma correcta
if predicted_mask_unet.ndim == 2:  # Si es 2D (alto, ancho)
    predicted_mask_unet = np.expand_dims(predicted_mask_unet, axis=-1)  # Agregar una dimensión de canal


## Visualización de resultados


In [ ]:
def display(display_list):
    title = ['Input Image', 'Ground Truth', 'Predicted Mask']
    plt.figure(figsize=(15, 5))
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

# Llame a la función de visualización
display([sample_image, sample_mask, predicted_mask_unet])


### Visualización para diferentes modelos


In [ ]:
display([sample_image, sample_mask, predicted_mask_resnet])
display([sample_image, sample_mask, predicted_mask_resnet_tf])
